# 4. Практика

Наша практика будет основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа).

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

[Скачать данные](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@_train_sem09__1_.zip)
 

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

### Описание задачи

- Необходимо обучить две модели: логистическую регрессию и случайный лес. 
- Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [2]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


%matplotlib inline
#plt.style.use('seaborn')
plt.style.use('seaborn-v0_8')

In [3]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.7


In [4]:
import optuna
print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 4.2.1


Загружаем данные

In [5]:
data = pd.read_csv('data/_train_sem09__1_.zip')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение). 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

****

### Логистическая регрессия

Зафиксируем только метрики, которые были получены без дополнительной настройки, т.е со значениями гиперпараметров, установленных по умолчанию:

In [8]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
y_train_pred = log_reg.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = log_reg.predict(X_test)
print('f1_score на наборе Test  : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.76
f1_score на наборе Train : 0.89
f1_score на наборе Test  : 0.78


### Случайный лес

Проделаем аналогичное для RandomForestClassifier().
Сначала посчитаем модель с параметрами по умолчанию и оценим метрику:

In [9]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('f1_score на наборе Test  : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на наборе Train : 1.00
f1_score на наборе Test  : 0.81


*****

### <center> **GridSearchCV**

#### GridSearchCV (estimator = LogisticRegression)  
включает в себя не только поиск лучших параметров, но и автоматическое построение новой модели на всем обучающем наборе данных, используя параметры, которые дают наилучшее значение точности при кросс-валидации.

In [ ]:
param_grid = [
    {
        'penalty': ['l2', None],  # для 'none' используем None
        'solver': ['lbfgs', 'sag'],  # эти солверы работают только с l2 или none
        'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
    },
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],  # эти солверы работают с l1 и l2
        'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
    }
]

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid,
    cv=5, 
    n_jobs = -1,
    error_score=0  # заменяем nan на 0
)  
%time grid_search.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test, y_test)))
y_train_pred = grid_search.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на наборе Test  : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))
print("Наилучшая модель:\n{}".format(grid_search.best_estimator_))

CPU times: total: 2.69 s
Wall time: 5min 42s
accuracy на тестовом наборе: 0.77
f1_score на наборе Train : 0.85
f1_score на наборе Test  : 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Наилучшая модель:
LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear')


#### GridSearchCV (estimator = RandomForestClassifier)  
включает в себя не только поиск лучших параметров, но и автоматическое построение новой модели на всем обучающем наборе данных, используя параметры, которые дают наилучшее значение точности при кросс-валидации.

In [13]:
param_grid = {'n_estimators': list(range(80, 200, 10)),
              'min_samples_leaf': [3, 7],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }
            
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5,    
    n_jobs = -1
)  
%time grid_search_forest.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
y_train_pred = grid_search_forest.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на наборе Test  : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))
print("Наилучшая модель:\n{}".format(grid_search_forest.best_estimator_))

CPU times: total: 4.55 s
Wall time: 1min 38s
accuracy на тестовом наборе: 0.81
f1_score на наборе Train : 0.98
f1_score на наборе Test  : 0.83
Наилучшие значения гиперпараметров: {'max_depth': np.int64(20), 'min_samples_leaf': 3, 'n_estimators': 110}
Наилучшая модель:
RandomForestClassifier(max_depth=np.int64(20), min_samples_leaf=3,
                       n_estimators=110, random_state=42)


****

### <center> **RandomizedSearchCV**

С использованием класса RandomizedSearchCV из библиотеки scikit learn мы осуществим оптимизацию гиперпараметров для алгоритмов логистической регрессии и случайного леса.

RandomizedSearchCV (estimator = LogisticRegression)

In [15]:
param_distributions = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 15, dtype=float))}
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search.score(X_test, y_test)))
y_train_pred = random_search.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search.predict(X_test)
print('f1_score на наборе Test : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))
print("Наилучшая модель:\n{}".format(random_search.best_estimator_))

c:\Users\stepu\OneDrive\IDE\.venv313\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
125 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\stepu\OneDrive\IDE\.venv313\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\stepu\OneDrive\IDE\.venv313\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\stepu\OneDrive\IDE\.venv313\Lib\site-packages\sk

CPU times: total: 3.98 s
Wall time: 1min 49s
accuracy на тестовом наборе: 0.77
f1_score на наборе Train : 0.85
f1_score на наборе Test : 0.79
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': np.float64(0.08071428571428571)}
Наилучшая модель:
LogisticRegression(C=np.float64(0.08071428571428571), max_iter=1000,
                   random_state=42)


RandomizedSearchCV (estimator = RandomForestClassifier)

In [17]:
param_distributions = {'n_estimators': list(range(80, 200, 20)),
              'min_samples_leaf': [5, 7],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
y_train_pred = random_search_forest.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на наборе Test  : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))
print("Наилучшая модель:\n{}".format(random_search_forest.best_estimator_))

CPU times: total: 2.41 s
Wall time: 42.9 s
accuracy на тестовом наборе: 0.80
f1_score на наборе Train : 0.94
f1_score на наборе Test  : 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 140, 'min_samples_leaf': 5, 'max_depth': np.int64(20)}
Наилучшая модель:
RandomForestClassifier(max_depth=np.int64(20), min_samples_leaf=5,
                       n_estimators=140, random_state=42)


****

### <center> Hyperopt

Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [27]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [28]:
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [29]:
%%time
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          #rstate=np.random.RandomState(random_state)
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [03:02<00:00,  3.65s/trial, best loss: -0.8124203668946883]
Наилучшие значения гиперпараметров {'max_depth': np.float64(20.0), 'min_samples_leaf': np.float64(2.0), 'n_estimators': np.float64(158.0)}
CPU times: total: 1min 23s
Wall time: 3min 2s


In [30]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_train_pred = model.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model.predict(X_test)
print('f1_score на наборе Test : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров {}".format(best))


accuracy на тестовом наборе: 0.81
f1_score на наборе Train : 0.99
f1_score на наборе Test : 0.83
Наилучшие значения гиперпараметров {'max_depth': np.float64(20.0), 'min_samples_leaf': np.float64(2.0), 'n_estimators': np.float64(158.0)}


****

## <center> Optuna

Optuna - это достаточно новый фреймворк/библиотека, разработанный специально для оптимизации гиперпараметров. Помимо байесовских алгоритмов, есть возможность удаления плохих комбинаций из рассмотрения. По умолчанию удаляет комбинации, в которых модель дает качество ниже медианы из уже рассмотренных. 

In [31]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, step=1)
  max_depth = trial.suggest_int('max_depth', 10, 30, step=1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, step=1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  
  # обучать модель можно также с помощью кросс-валидации
  # применим  cross validation с тем же количеством фолдов
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [32]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)

[I 2025-03-14 18:28:53,111] A new study created in memory with name: RandomForestClassifier
[I 2025-03-14 18:28:57,751] Trial 0 finished with value: 0.7998612199144762 and parameters: {'n_estimators': 197, 'max_depth': 24, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7998612199144762.
[I 2025-03-14 18:29:01,124] Trial 1 finished with value: 0.8095374098065624 and parameters: {'n_estimators': 120, 'max_depth': 18, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.8095374098065624.
[I 2025-03-14 18:29:04,020] Trial 2 finished with value: 0.8087816538512864 and parameters: {'n_estimators': 110, 'max_depth': 25, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.8095374098065624.
[I 2025-03-14 18:29:07,037] Trial 3 finished with value: 0.8063744851766106 and parameters: {'n_estimators': 109, 'max_depth': 19, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.8095374098065624.
[I 2025-03-14 18:29:10,909] Trial 4 finished with value: 0.8081544674803365 and parameters: {'n_

CPU times: total: 1min 30s
Wall time: 3min 40s


In [35]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

Наилучшие значения гиперпараметров {'n_estimators': 103, 'max_depth': 28, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.82


In [34]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
y_train_pred = model.predict(X_train)
print('f1_score на наборе Train : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model.predict(X_test)
print('f1_score на наборе Test : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

accuracy на тестовом наборе: 0.80
f1_score на наборе Train : 0.99
f1_score на наборе Test : 0.82


***

## <center> ИТОГИ

В результате выполнения задания получены следующие результаты:

1. Логистическая регрессия
  - f1_score на наборе Train : 0.89
  - f1_score на наборе Test  : 0.78
2. Случайный лес
  - f1_score на наборе Train : 1.00
  - f1_score на наборе Test  : 0.81
3. GridSearchCV (estimator = LogisticRegression) 
  - f1_score на наборе Train : 0.85
  - f1_score на наборе Test  : 0.79
  - Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
  - Наилучшая модель: LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear')
4. GridSearchCV (estimator = RandomForestClassifier) 
  - f1_score на наборе Train : 0.98
  - f1_score на наборе Test  : 0.83
  - Наилучшие значения гиперпараметров: {'max_depth': np.int64(20), 'min_samples_leaf': 3, 'n_estimators': 110}
  - Наилучшая модель: RandomForestClassifier(max_depth=np.int64(20), min_samples_leaf=3,
                       n_estimators=110, random_state=42)
5. RandomizedSearchCV (estimator = LogisticRegression) 
  - f1_score на наборе Train : 0.85
  - f1_score на наборе Test : 0.79
  - Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': np.float64(0.08071428571428571)}
  - Наилучшая модель: LogisticRegression(C=np.float64(0.08071428571428571), max_iter=1000,
6. RandomizedSearchCV (estimator = RandomForestClassifier) 
  - f1_score на наборе Train : 0.94
  - f1_score на наборе Test  : 0.82
  - Наилучшие значения гиперпараметров: {'n_estimators': 140, 'min_samples_leaf': 5, 'max_depth': np.int64(20)}
  - Наилучшая модель: RandomForestClassifier(max_depth=np.int64(20), min_samples_leaf=5,
                       n_estimators=140, random_state=42)
7. Hyperopt с модельюRandomForestClassifier
  - f1_score на наборе Train : 0.99
  - f1_score на наборе Test : 0.83
  - Наилучшие значения гиперпараметров {'max_depth': np.float64(20.0), 'min_samples_leaf': np.float64(2.0), 'n_estimators': np.float64(158.0)}

8. Optuna с моделью RandomForestClassifier
  - Наилучшие значения гиперпараметров {'n_estimators': 150, 'max_depth': 25, 'min_samples_leaf': 5}
  - f1_score на наборе Train : 0.99
  - f1_score на наборе Test : 0.82
  - Наилучшие значения гиперпараметров {'n_estimators': 103, 'max_depth': 28, 'min_samples_leaf': 2}


***Лучшую метрику f1_score на наборе Test (0.83) показал два метода:***
- GridSearchCV с моделью RandomForestClassifier со значениями гиперпараметров: {'max_depth': np.int64(20), 'min_samples_leaf': 3, 'n_estimators': 110}
- Hyperopt с моделью RandomForestClassifier со значениями гиперпараметров: {'max_depth': np.float64(20.0), 'min_samples_leaf': np.float64(2.0), 'n_estimators': np.float64(158.0)}


Кроме того, можно отметить, что в данном задании подбора гиперпараметров с помощью базовых и продвинутых методов оптимизации модель RandomForestClassifier показывает в целом лучшие значения метрики, чем LogisticRegression.
